# Implementation: PyTorch + PennyLane Integration

**Goal**: Hybrid AI.

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn

# 1. Define Quantum Node
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    # Encode Input Data
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    # Trainable Quantum Layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# 2. Convert to PyTorch Layer
weight_shapes = {"weights": (3, n_qubits, 3)}
q_layer = qml.qnn.TorchLayer(qnode, weight_shapes)

# 3. Define Hybrid Model
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.q_layer = q_layer  # Quantum Sandwich
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.q_layer(x)
        x = self.fc2(x)
        return x

model = HybridModel()
print(model)
print("You can now train this with standard torch.optim.Adam!")

## Conclusion
The future is Hybrid.